In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os, sys
sys.path.append('..')
from glob import glob
from configs import *
from tqdm import tqdm
from utils import *
import numpy as np
import matplotlib.pyplot as plt
from dataset import *
from time import time
tqdm.monitor_interval = 0
import torch
import json
import torch.nn as nn
import numpy as np
from time import time, strptime, mktime
torch.manual_seed(0)
np.random.seed(0)

In [3]:
dfiles = sorted(glob('../data/next_stops/*.json'))
print(len(dfiles))

90


In [53]:
all_adjs = {}
for fname in tqdm(dfiles):
    with open(fname) as fl:
        data = json.load(fl)
    
#     print(len(data))
    for rid, byroute in data.items():
        dir, _, rname = rid.split('_')
        rid = '%s_%s' % (rname.strip('+'), dir)
        if rid not in all_adjs: all_adjs[rid] = {}
        for sid, nexts in byroute.items():
            if sid not in all_adjs[rid]: all_adjs[rid][sid] = {}
            for next_stop, occur in nexts.items():
                if next_stop not in all_adjs[rid][sid]: all_adjs[rid][sid][next_stop] = 0
                all_adjs[rid][sid][next_stop] += occur
#     break

100%|██████████| 90/90 [00:04<00:00, 18.85it/s]


In [54]:
sfiles = sorted(glob('../data/stopcodes_sequence/*.txt'))
print(len(sfiles))

656


In [65]:
raw = lambda ent: 'MTA_' + ent

In [85]:
stopcodes = {}
for fname in sfiles:
    with open(fname) as fl:
        stops = fl.read().split('\n')
    stops = [stop for stop in stops if stop]
    stopcodes[fileName(fname)] = stops

In [86]:
obs_stats = {}
for fname, route_hist in all_adjs.items():
    if fname not in stopcodes: 
#         print('Missing:', fname)
        continue
#     print(fname)
    known = stopcodes[fname]
    
    no_stop = 0
    no_seq = 0
    for si in range(1, len(known)):
        st, ed = known[si-1], known[si]
        if raw(st) not in route_hist:
            no_stop += 1
        elif raw(st) in route_hist:
            if raw(ed) not in route_hist[raw(st)]:
                no_seq += 1
    no_stop = no_stop / len(known)
    no_seq = no_seq / len(known)
    obs_stats[fname] = (no_stop, no_seq, len(known))
#     print(fname, no_stop, no_seq)

In [87]:
okeys = list(obs_stats.keys())
okeys = sorted(okeys, key=lambda k: (obs_stats[k][0] * 100 + obs_stats[k][1]))

In [143]:
complete = []
almost = []
for rname in reversed(okeys):
#     if rname[0] != 'M':
#         continue
    no_stop, no_seq, nk = obs_stats[rname]
    if no_stop == 0: complete.append(rname)
    if no_stop < 0.15: almost.append(rname)
    print('%.1f%% %.1f%% :  [%d] %s' % (no_stop*100, no_seq*100, nk, rname))

46.2% 0.0% :  [13] M35_1
28.6% 7.1% :  [28] Q48_1
23.8% 4.8% :  [21] M22_0
19.2% 7.7% :  [26] QM12_1
18.6% 11.6% :  [43] B37_0
17.5% 12.5% :  [40] B37_1
17.5% 5.0% :  [40] Q47_1
14.8% 7.4% :  [27] QM12_0
14.8% 3.7% :  [27] Q72_1
14.3% 14.3% :  [21] M12_1
14.3% 7.1% :  [42] Q47_0
13.0% 4.3% :  [23] Q48_0
12.5% 7.5% :  [40] Q67_0
11.9% 9.5% :  [42] M20_0
11.8% 7.8% :  [51] Q65_0
11.5% 11.5% :  [26] QM11_0
11.5% 7.7% :  [26] QM11_1
11.5% 7.7% :  [26] QM25_1
11.5% 7.7% :  [26] QM25_0
11.1% 7.4% :  [27] QM20_0
10.3% 13.8% :  [29] M14D_0
10.1% 1.4% :  [69] M1_0
10.0% 10.0% :  [20] M12_0
10.0% 5.0% :  [20] Q52_1
9.8% 2.0% :  [51] Q23_0
9.5% 9.5% :  [21] QM7_1
9.5% 9.5% :  [21] QM7_0
9.4% 21.9% :  [32] QM2_0
9.5% 4.8% :  [21] Q104_1
8.9% 4.4% :  [45] Q32_1
8.7% 2.9% :  [69] M1_1
8.3% 4.2% :  [24] Q53_1
7.8% 2.0% :  [51] B35_1
7.7% 7.7% :  [13] B84_1
7.7% 0.0% :  [13] M96_1
7.6% 7.6% :  [79] Q27_1
7.4% 7.4% :  [27] QM3_0
6.9% 27.6% :  [29] Q52_0
6.7% 6.7% :  [30] M31_0
6.7% 6.7% :  [15] M42_1
6

In [120]:
print('     Complete: %d/%d' % (len(complete), len(okeys)))
print('Almost (>85%%): %d/%d' % (len(almost), len(okeys)))

     Complete: 40/428
Almost (>85%): 64/428


In [126]:
rname = complete[2]
rname

'M11_0'

In [132]:
def greedy_skip(stops, nexts):

    # for si in range(1, 10):
    si = 1
    skipped = []
    occ = 0
    while si < len(stops):
        st, ed = stops[si-1], stops[si]
    #     if raw(st) in nexts:
        skipped.append(st)
        if raw(ed) not in nexts[raw(st)]:
            observed = nexts[raw(st)]
    #             print('Next stop not observed:')
    #             print(si, st, ed)
    #             print(observed)
            smax = max(list(observed.keys()), key=lambda k: observed[k])[4:]
    #             print('Best next:', smax)
    #             print('Observed:')
            occ += observed[raw(smax)]
            for ri in range(si, len(stops)):
    #                 print(stops[ri], nexts[raw(stops[ri])])
                if stops[ri] == smax:
                    break
            assert ri < len(stops)
            si = ri + 1
            continue
        si += 1
    matched = len(skipped)/ len(stops)
    
    return skipped, matched
    # print()
stops = stopcodes[rname]
nexts = all_adjs[rname]
skipped, matched = greedy_skip(stops, nexts)
print(rname, '%.2f' % matched)

M11_0 0.97


In [133]:
def show_matches(skipped, stops):
    ii = 0
    for si, stop in enumerate(stops):

        ci = ii
        while ci < len(skipped):
            if stop == skipped[ci]:
                break
            ci += 1

        if ci == len(skipped):
            # not found
            print(si, stop)
            continue
        else:
            print(si, stop, skipped[ci])
            ii = ci
show_matches(skipped, stops)

0 401393 401393
1 405420 405420
2 405265 405265
3 405152 405152
4 405151 405151
5 404161 404161
6 401398 401398
7 403802 403802
8 401400 401400
9 401401 401401
10 401402 401402
11 401404 401404
12 404279 404279
13 401407 401407
14 401408 401408
15 401409 401409
16 401410 401410
17 401411 401411
18 401412 401412
19 401413 401413
20 401414 401414
21 401416 401416
22 401417 401417
23 403472 403472
24 403555 403555
25 401021 401021
26 401022 401022
27 401023 401023
28 401024 401024
29 401025 401025
30 401026 401026
31 401027 401027
32 404320 404320
33 401029 401029
34 401030 401030
35 401031 401031
36 401032 401032
37 401033 401033
38 401034 401034
39 401035 401035
40 401435 401435
41 401436 401436
42 401438 401438
43 403886 403886
44 401440 401440
45 401441 401441
46 401442 401442
47 401443 401443
48 403292 403292
49 401444 401444
50 401446 401446
51 401447 401447
52 403757 403757
53 404011 404011
54 404012 404012
55 404013 404013
56 404056 404056
57 404014 404014
58 404015 404015
59 4041

In [142]:
for rname in complete:
    stops = stopcodes[rname]
    nexts = all_adjs[rname]
    skipped, mscore = greedy_skip(stops, nexts)
#     print('%.1f%%' % (mscore * 100), rname, '[%d]' % len(stops))
    with open('../data/common_stops/%s.txt' % rname, 'w') as fl:
        fl.write('\n'.join(skipped))
#     show_matches(skipped, stops)
#     break